Idea and Hypothesis:
When evolving the base CTRNN state and the HP mechanism together to perform well at various temperature values (arbitrary global perturbation), we expect two non-mutually exclusive strategies to emerge

1) Move the base state to an area of parameter space where temperature variation induces a relatively mild fitness dropoff (is aligned with a degenerate fitness ridge)

2) Identify an HP mechanism which activates in response to temperature changes and restores parameters to a fit location

2b) Identify an HP mechanism which, when seeded with the appropriate base state, drives an oscillation that is reasonably fit and maintains this oscillation despite temperature variation. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from CTRNNclass import *
from acceptance import *
from HPevolution import *
from pyloricfitness import *
from CTRNNgenomesamp import *

### Additional thoughts:
- How do these things depend on the nature of the perturbation? 

    - Variations to test: (1) Ordered global direciton vs. random noise (makes impossible to align to degenerate edge), (2) Different ordered global directions may be easier or harder to find edges corresponding to them and/or HP mechanisms suited to them and/or HP mechanisms which can continue to drive appropriate oscillations under their conditions
    
- How do these things depend on the nature of the task?

    - For example, with the walker, maybe degenerate ridges are easier to find, or maybe HP mechanisms are easier to find, or maybe this HP formulation is just better suited to the task (i.e. sliding window can make the shape, order doesn't matter) 
    
    - What about a behaving circuit that needs to respond to stimuli appropriately (not a CPG)? Then, maybe HP is a less viable option (disrupts appropriate responses). Maybe this could be remedied with slowing it down (separation of timescales), but how much is reasonable? And if not, this would be a major hit to the idea of HP because true CPG's do not exist. 
    
- How do these things depend on the nature of HP?

    - Differential application, sliding window, Hywell Williams vs. DiPaolo protocols, ...
    
    - Including a pacemaker element?
    
    - Dealing with time constants - should they reasonably be affected by temperature? should they reasonably be under the control of HP? What would it mean for animals if their HP had or didn't have control over them?
    
    - Demonstrated that HP needs capping so that weights don't go to zero with biases exploding - think about why that is

- How do these things depend on the details of evolution?

    - Evolving basal state first and then HP (or vice versa) vs. concurrently 
    
    - Differently weighting the importance of performing under basal state vs. performing under perturbation
    
    - Crossover and mutation differences (keeping basal state and HP as separate units)
    
- How would an experimentalist observe this?

    - Crabs pulled from the environment with variability along the degenerate fitness ridges? Response bias or evolution at work?
        - Different arrangement when the perturbations have been larger/more disruptive? (i.e. climate change)
        - Observe correlational structure even though none is "built in" to the HP mechanisms? Or even when no HP mechanism is present at all
    
    - 

In [2]:
###### HP sample params ###########
taubbounds = [10,30]
tauwbounds = [30,50]
slidingwindowbounds = [1,100]

def randomHPsample(CTRNNsize, samplesize):
    genomes = np.zeros((samplesize,(CTRNNsize*2) + 3))
    for i in range(samplesize): #first, set random bias time constant, weight time constant, and sliding window length
        HPgenome = np.array([1,1,1,0,0,0,np.random.uniform(low=tauwbounds[0],high=tauwbounds[1]),np.random.uniform(low=taubbounds[0],high=taubbounds[1]),np.random.randint(low=slidingwindowbounds[0],high=slidingwindowbounds[1])])
        for j in range(CTRNNsize): #for every neuron, set a lower and upper bound
            while HPgenome[j] > .5:
                HPgenome[j] = 1-np.random.power(1)  #could have just had uniform distributuion like the rest, but I felt the need to sample the most solutions with wide target ranges
            while HPgenome[j+CTRNNsize] < .5:
                HPgenome[j+CTRNNsize] = np.random.power(1)
        genomes[i] = HPgenome
    return genomes

###### CTRNN base state sample params ########
wtbounds = [-16,16]
biasbounds = [-16,16]
taubounds = [.5,10]
n = 3

def randomCTRNNsample(CTRNNsize,samplesize):
    n = CTRNNsize
    genelength = (n**2)+(2*n)
    genomes = np.zeros((samplesize,genelength))
    for i in range(samplesize):
        wts = np.random.uniform(low = wtbounds[0],high = wtbounds[1],size = (n**2))
        biases = np.random.uniform(low = biasbounds[0],high = biasbounds[1],size = n)
        taus = np.random.uniform(low = taubounds[0],high = taubounds[1],size = n)
        CTRNNgenome = np.concatenate((wts,biases,taus))
        genomes[i] = CTRNNgenome
    return genomes

In [3]:
#a pyloric-like neurongenome evolved from the other notebook
#in case ever needed

sol1 = np.array([ 14.82024187,  10.61253584,-12.15141158,  -4.61967513,-15.81543225,  15.62745189,  14.71262429,   2.0523492 , 1.86045721, -10.25813196,  -0.29218389,  -2.47607644, 1.13091869,   4.00518322,   3.57617411])

In [9]:
##### Global perturbation (temperature) params #######
n = 3
freepars = (n**2) + n  #number of pars that will be perturbed by temperature (not time constants)
globalpertdirection = np.random.uniform(-1,1,size=freepars) 
pertvector = (globalpertdirection/np.linalg.norm(globalpertdirection)) #normalized to have length 1
globalpertmagnitude = 1 #length it will actually have in practice
pts = 3 #number of locations to test along the pertvector

print(pertvector)

def fitnessfunc(genome):
    '''genome has the form [weights,biases,timeconsts,lbs,ubs,taub,tauw,slidingwindow]. 
    pertvector and the number of sample points along it is defined outside the function
    Apply perts of different magnitudes to the homeostatic CTRNN and average its pyloric fitness @ each point'''
    neurongenome = genome[:-((2*n)+3)]
    HPgenome = genome[-((2*n)+3):]
    fitnesssum = 0
    for i in np.linspace(0,globalpertmagnitude,pts): 
        neurongenome[:len(pertvector)] += (i*pertvector)
        fitnesssum += pyloriclike(neurongenome,HPgenome)
    return (fitnesssum/pts)

[ 0.2919818  -0.19653026 -0.3824069  -0.28102191  0.17555101 -0.15973807
 -0.16450977 -0.1198932   0.47956992  0.50142158 -0.26209938 -0.05508989]


In [5]:
#####Microbial Genetic Algorithm Params ###############
CTRNNsize = 3
popsize = 15
startpopulation = np.concatenate((randomCTRNNsample(CTRNNsize,popsize),randomHPsample(CTRNNsize,popsize)),axis=1)
print('example genome', startpopulation[0])
recombProb = .5 
mutatProb = .25 #really, its the magnitude of mutation
generations = 200
differentialapp = [1,1,1] #HP can apply itself to any neuron

M = Microbial(fitnessfunc, startpopulation, recombProb, mutatProb, generations, differentialapp)

example genome [ -1.74210778 -10.32777833 -12.30614074   5.63527595  -2.3767056
  13.83257816   5.36861722 -15.87031069   7.99261014 -10.94222297
  -4.84435278  -6.70680397   5.4132097    1.77899558   3.48372781
   0.31569187   0.06065861   0.06721901   0.53846188   0.75036543
   0.81917537  30.68826521  15.65602958  14.        ]


In [6]:
M.run()
M.showFitness()

#must increase transients for a longer HP process
#investigae all the overflows
#doubleperiodicity much more common with HP, or at least the appearance of it from a 3D projection

0
0.6949110314078943
unable to find two full cycles
CTRNN [  1.34120254   0.29047698 -15.04462598  -1.922432     3.97382553
   0.92644559 -12.76385669  -9.21532031 -15.04462598   2.89919334
   0.8223705    3.50208161   6.57542161   1.4011577    3.11590776]
HP [1.11710357e-02 7.53457389e-01 9.72619632e-02 8.81430010e-01
 7.58537240e-01 1.07261963e-01 3.28417598e+01 1.34210556e+01
 7.10000000e+01]
possible double-periodicity
CTRNN [-2.62117468  5.13543631 -2.00799153  0.89852185  5.67199857  2.36428679
  6.45017446 -0.53924011  6.11289317 -1.81758301 -5.80805946 -2.36881167
  8.34466354  5.71699943  0.37443606]
HP [ 0.39450445  0.24823514  0.24188738  0.40450445  0.25823514  0.35077018
 45.89937552 12.04251646  1.        ]
possible double-periodicity
CTRNN [-2.70978951  5.62532721 -1.85162405  1.01522144  5.98091434  2.67975183
  6.66327002 -0.26415243  6.48455036 -1.6094542  -5.3929414  -2.3208986
  8.34466354  5.71699943  0.37443606]
HP [ 0.39450445  0.24823514  0.24188738  0.40450445 

C:\Users\LJSbo\Pyloric Rhythm CTRNNs\CTRNNclass.py:7: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
C:\Users\LJSbo\Pyloric Rhythm CTRNNs\CTRNNclass.py:117: RuntimeWarning: overflow encountered in multiply
  self.States += self.dt * (self.invTimeConstants*(-self.States+netinput))


unable to find two full cycles
CTRNN [-3.09270304e+00  3.03449363e-01  8.41792398e-04  5.05693690e+00
  2.52463889e+00  8.21946640e-04 -1.60000000e+01 -1.60000000e+01
  8.28216191e-04  5.60389314e-01 -1.14334345e+00 -5.00391264e+00
  8.93361117e+00  1.42302106e+00  2.84238854e+00]
HP [-0.89686692  0.28944066 -0.1436959   1.05418957  0.65324891 -0.1336959
 39.85565502 18.69094266 97.        ]


KeyboardInterrupt: 

In [8]:
print(M.fitStats())

(0.1605865410764879, 0.7326001666055033, array([-1.66277628e+00,  6.08372573e+00,  2.02550211e-02,  9.42733663e-01,
        8.46697321e+00,  4.99193437e+00, -5.60691553e+00, -6.66479592e+00,
        5.06919151e+00,  3.36623523e+00, -3.18997036e+00, -4.84189892e+00,
        7.37470557e+00,  9.79376709e+00,  2.56164698e+00,  3.58472612e-01,
        2.09458835e-01,  3.42017940e-01,  3.68472612e-01,  2.19458835e-01,
        8.28364889e-01,  3.99910632e+01,  1.90137781e+01,  4.00000000e+01]))
